In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
book_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0")
book_testparquet

In [ ]:
trade_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0")
trade_testparquet

#  train.csv

In [ ]:
train = pd.read_csv("../input/optiver-realized-volatility-prediction/train.csv")
train

#  train.parquet

In [ ]:
book_example = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
book_example

In [ ]:
trade_example = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0")
trade_example

##  Analysis by stock
###  Number of shares

In [ ]:
train

In [ ]:
for col in train.columns:
    print(col,":",len(train[col].unique()))

### There are 112 types of stock_id, 3830 types of time_id, and 414287 types of target.

##  Statistic by stock

In [ ]:
stock = train.groupby("stock_id")["target"].agg(["mean","median","std","count","sum"]).reset_index()
stock

In [ ]:
print("mean value=" ,stock["mean"].mean())
plt.hist(stock["mean"])
plt.show()

In [ ]:
print("sum value=" ,stock["sum"].mean())
plt.hist(stock["sum"])
plt.show()

## Relationship between Bid, Ask on the order book, and actual transaction behavior price within time_id

In [ ]:
book_example

In [ ]:
book_test = book_example[book_example["time_id"]==5]
book_test

##  Price fluctuation (individual + whole)

#### First, order book information

In [ ]:
samples = ["bid_price1","bid_price2","ask_price1","ask_price2"]

for num,a in enumerate(samples):
    plt.figure(figsize=(20,5))
   
    plt.subplot(4,1,num+1)
    plt.plot(book_test["seconds_in_bucket"],book_test[a])
    plt.title(a)
plt.show()
plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    
   
    plt.plot(book_test["seconds_in_bucket"],book_test[a],label=a)
plt.legend(fontsize=12)


#### Add the actual transaction information to this.

In [ ]:
trade_example

In [ ]:
trade_test = trade_example[trade_example["time_id"]==5]
trade_test.head(5)

#### Add the actual transaction to the whole graph

In [ ]:
plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    
   
    plt.plot(book_test["seconds_in_bucket"],book_test[a],label=a)
    
plt.plot(trade_test["seconds_in_bucket"],trade_test["price"],label="trade_parquet",lw=10)
plt.legend(fontsize=12)

### Purple is the actual deal. It's wandering between bid and ask on the order book.

## One of the image for understanding the volatility

Let's visualize when the volatility is the lowest of stock_id=0.

In [ ]:
stock0 = train[train["stock_id"]==0]
min_index = stock0["target"].idxmin()
min_time_id = stock0.iloc[min_index]["time_id"]
print("min index is",min_time_id,"min target is",stock0.iloc[min_index]["target"])

In [ ]:
book_test_min = book_example[book_example["time_id"]==min_time_id]
trade_test_min = trade_example[trade_example["time_id"]==min_time_id]


plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    
   
    plt.plot(book_test_min["seconds_in_bucket"],book_test_min[a],label=a)
    
plt.plot(trade_test_min["seconds_in_bucket"],trade_test_min["price"],label="trade_parquet",lw=10)
plt.legend(fontsize=12)

On the other hand, let's visualize when the volatility is the highest of stock_id=0.

In [ ]:
stock0 = train[train["stock_id"]==0]
max_index = stock0["target"].idxmax()
max_time_id = stock0.iloc[max_index]["time_id"]
print("max index is",max_time_id,"max target is",stock0.iloc[max_index]["target"])

In [ ]:
book_test_max = book_example[book_example["time_id"]==max_time_id]
trade_test_max = trade_example[trade_example["time_id"]==max_time_id]


plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    
   
    plt.plot(book_test_max["seconds_in_bucket"],book_test_max[a],label=a)
    
plt.plot(trade_test_max["seconds_in_bucket"],trade_test_max["price"],label="trade_parquet",lw=10)
plt.legend(fontsize=12)

Since the scale of the vertical axis is completely different, I compared the actual trade transactions (purple lines).

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(trade_test_min["seconds_in_bucket"],trade_test_min["price"],lw=10,label="min_vol_time")
plt.plot(trade_test_max["seconds_in_bucket"],trade_test_max["price"],lw=10,label = "max_vol_time")
plt.legend(fontsize=15)

When the volatility is high, in this case, it can be seen that the price fluctuates considerably in 10 minutes.


(Variation and Max-Min seem to be important)


This is one of the examples for the image of the volatility.

### submit

#### In the trial, enter all the median values ​​for each stock and submit

#### Creating a dictionary of median values for each stock

In [ ]:
stock

In [ ]:
stock2 = stock[["stock_id","median"]]
stock2 = stock2.set_index("stock_id")
stock2


In [ ]:
stock_dict = stock2.to_dict()

# example : stock id = 0 median median value
stock_dict["median"][0]

### Replace submit file and generate submission file

In [ ]:
sample = pd.read_csv("../input/optiver-realized-volatility-prediction/sample_submission.csv")
sample


In [ ]:
test = pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test

In [ ]:
sample 

Extract stock_id from row_id

In [ ]:
sample["stock_id"] = [s.split("-")[0] for s in sample["row_id"]]
sample

Substitute the contents of the dictionary type

In [ ]:
sample["target"] = [stock_dict["median"][int(s)] for s in sample["stock_id"]]
sample

stock_id deleted

In [ ]:
sample = sample.drop("stock_id",axis=1)
sample

In [ ]:
sample.to_csv("submission.csv",index=False)